# Knowledge Graph
---

In [6]:
import json
import hashlib
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, XSD, FOAF, DCTERMS
from urllib.parse import quote
import os
from datetime import datetime
from typing import Dict, List, Tuple

def create_knowledge_graph(json_file_path: str, output_dir: str) -> Dict[str, str]:
    """
    Creates a knowledge graph from a JSON knowledge representation file.
    
    Args:
        json_file_path: Path to the JSON knowledge base file
        output_dir: Directory to save the output knowledge graph files
        
    Returns:
        Dictionary with paths to the generated files
    """
    # Load the JSON knowledge base
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            knowledge_base = json.load(f)
    except Exception as e:
        raise ValueError(f"Failed to load JSON file: {str(e)}")
    
    # Create RDF Graph
    g = Graph()
    
    # Define namespaces - FIXED: Properly define all namespaces
    ns = Namespace("http://example.org/chinese-culture/")
    dbo = Namespace("http://dbpedia.org/ontology/")
    schema = Namespace("http://schema.org/")
    prov = Namespace("http://www.w3.org/ns/prov#")
    
    g.bind("ns", ns)
    g.bind("dbo", dbo)
    g.bind("schema", schema)
    g.bind("prov", prov)
    
    # Create a dictionary to map predicates to more semantic relationships
    PREDICATE_MAPPING = {
        "related_to": ns.relatedTo,
        # Add more specific mappings as needed
        # "born_in": schema.birthPlace,
        # "died_in": schema.deathPlace,
    }
    
    # Track entities and their types for better classification
    entity_types = {}
    
    # Process each document chunk
    for chunk in knowledge_base:
        # Create document entity with provenance information
        doc_uri = ns[f"document/{quote(chunk['id'])}"]
        g.add((doc_uri, RDF.type, schema.Article))
        g.add((doc_uri, DCTERMS.source, Literal(chunk['source'])))
        g.add((doc_uri, schema.text, Literal(chunk['text'], lang="ms")))
        g.add((doc_uri, DCTERMS.created, Literal(datetime.now().isoformat(), datatype=XSD.dateTime)))
        
        # Process triples
        for triple in chunk['triples']:
            if len(triple) != 3:
                continue  # Skip malformed triples
                
            subject = triple[0].strip()
            predicate = triple[1].strip()
            obj = triple[2].strip()
            
            # Skip empty components
            if not subject or not predicate or not obj:
                continue
                
            # Create URIs for subject and object
            subject_uri = ns[f"entity/{quote(subject)}"]
            obj_uri = ns[f"entity/{quote(obj)}"]
            
            # Add subject and object as entities with labels
            g.add((subject_uri, RDF.type, dbo.Thing))
            g.add((subject_uri, RDFS.label, Literal(subject, lang="ms")))
            
            g.add((obj_uri, RDF.type, dbo.Thing))
            g.add((obj_uri, RDFS.label, Literal(obj, lang="ms")))
            
            # Try to infer types based on content
            infer_entity_type(subject_uri, subject, g, ns, dbo)
            infer_entity_type(obj_uri, obj, g, ns, dbo)
            
            # Add relationship between them
            rel_uri = PREDICATE_MAPPING.get(predicate, ns[predicate])
            g.add((subject_uri, rel_uri, obj_uri))
            
            # Add inverse relationship when appropriate
            if predicate == "related_to":
                g.add((obj_uri, rel_uri, subject_uri))
            
            # Link document to the entities
            g.add((doc_uri, schema.about, subject_uri))
            g.add((doc_uri, schema.about, obj_uri))
            g.add((subject_uri, prov.wasDerivedFrom, doc_uri))
            g.add((obj_uri, prov.wasDerivedFrom, doc_uri))
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate output filenames
    base_filename = os.path.splitext(os.path.basename(json_file_path))[0]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save in multiple formats
    output_files = {}
    
    # Turtle format
    ttl_path = os.path.join(output_dir, f"{base_filename}_kg_{timestamp}.ttl")
    g.serialize(destination=ttl_path, format='turtle')
    output_files['turtle'] = ttl_path
    
    # JSON-LD format
    jsonld_path = os.path.join(output_dir, f"{base_filename}_kg_{timestamp}.jsonld")
    g.serialize(destination=jsonld_path, format='json-ld')
    output_files['jsonld'] = jsonld_path
    
    # N-Triples format
    nt_path = os.path.join(output_dir, f"{base_filename}_kg_{timestamp}.nt")
    g.serialize(destination=nt_path, format='nt')
    output_files['ntriples'] = nt_path
    
    # Create a simplified JSON representation
    simplified_kg = create_simplified_kg(g, knowledge_base, schema, dbo, prov)
    simplified_path = os.path.join(output_dir, f"{base_filename}_kg_simplified_{timestamp}.json")
    with open(simplified_path, 'w', encoding='utf-8') as f:
        json.dump(simplified_kg, f, indent=2, ensure_ascii=False)
    output_files['simplified'] = simplified_path
    
    print(f"\nKnowledge graph creation complete:")
    print(f"- Entities: {len(simplified_kg['entities'])}")
    print(f"- Relationships: {len(simplified_kg['relationships'])}")
    print(f"- Source documents: {len(simplified_kg['documents'])}")
    print(f"\nGenerated files:")
    for fmt, path in output_files.items():
        print(f"- {fmt.upper()}: {path}")
    
    return output_files

def infer_entity_type(entity_uri, entity_name: str, graph: Graph, ns: Namespace, dbo: Namespace) -> None:
    """
    Attempts to infer a more specific type for an entity based on its name.
    """
    # Simple type inference based on patterns in the name
    lower_name = entity_name.lower()
    
    if any(x in lower_name for x in ['festival', 'perayaan']):
        graph.add((entity_uri, RDF.type, ns.Festival))
    elif any(x in lower_name for x in ['dynasty', 'dinasti']):
        graph.add((entity_uri, RDF.type, dbo.HistoricalPeriod))
    elif any(x in lower_name for x in ['city', 'bandar', 'kota']):
        graph.add((entity_uri, RDF.type, dbo.City))
    elif any(x in lower_name for x in ['country', 'negara']):
        graph.add((entity_uri, RDF.type, dbo.Country))
    elif any(x in lower_name for x in ['person', 'orang', 'tokoh']):
        graph.add((entity_uri, RDF.type, dbo.Person))
    elif any(x in lower_name for x in ['culture', 'budaya']):
        graph.add((entity_uri, RDF.type, dbo.Culture))
    # Add more type inference rules as needed

def create_simplified_kg(graph: Graph, original_kb: List[Dict], schema: Namespace, dbo: Namespace, prov: Namespace) -> Dict:
    """
    Creates a simplified JSON representation of the knowledge graph.
    """
    simplified = {
        "metadata": {
            "generated_at": datetime.now().isoformat(),
            "source": "Generated from knowledge_representation.json",
            "languages": ["ms"],  # Malay language
            "statistics": {}
        },
        "documents": [],
        "entities": [],
        "relationships": []
    }
    
    # Process documents
    doc_uris = set(graph.subjects(RDF.type, schema.Article))
    simplified['metadata']['statistics']['documents'] = len(doc_uris)
    
    for doc_uri in doc_uris:
        doc_info = {
            "uri": str(doc_uri),
            "source": str(graph.value(doc_uri, DCTERMS.source)),
            "text": str(graph.value(doc_uri, schema.text)),
            "entities": []
        }
        simplified['documents'].append(doc_info)
    
    # Process entities
    entity_uris = set(graph.subjects(RDF.type, None))
    simplified['metadata']['statistics']['entities'] = len(entity_uris)
    
    for entity_uri in entity_uris:
        # Skip documents
        if (entity_uri, RDF.type, schema.Article) in graph:
            continue
            
        labels = list(graph.objects(entity_uri, RDFS.label))
        types = [str(t) for t in graph.objects(entity_uri, RDF.type) if t != dbo.Thing]
        
        entity_info = {
            "uri": str(entity_uri),
            "label": str(labels[0]) if labels else "",
            "types": types,
            "source_documents": list(set(
                str(doc) for doc in graph.objects(entity_uri, prov.wasDerivedFrom)
            ))
        }
        simplified['entities'].append(entity_info)
        
        # Link entities to documents
        for doc in entity_info['source_documents']:
            for doc_info in simplified['documents']:
                if doc_info['uri'] == doc:
                    doc_info['entities'].append(str(entity_uri))
    
    # Process relationships
    rel_count = 0
    for s, p, o in graph:
        # Only include relationships between entities (not documents)
        if (s, RDF.type, schema.Article) in graph or (o, RDF.type, schema.Article) in graph:
            continue
            
        s_label = str(next(graph.objects(s, RDFS.label), s))
        o_label = str(next(graph.objects(o, RDFS.label), o))
        
        rel_info = {
            "subject": str(s),
            "subject_label": s_label,
            "predicate": str(p),
            "object": str(o),
            "object_label": o_label
        }
        simplified['relationships'].append(rel_info)
        rel_count += 1
    
    simplified['metadata']['statistics']['relationships'] = rel_count
    
    return simplified

if __name__ == "__main__":
    # Example usage
    input_json = "Knowledge representation/knowledge_representation.json"
    output_dir = "Knowledge representation/knowledge_graphs"
    
    try:
        result_files = create_knowledge_graph(input_json, output_dir)
        print("\nKnowledge graph created successfully!")
    except Exception as e:
        print(f"\nError creating knowledge graph: {str(e)}")
        import traceback
        traceback.print_exc() 

c:\Users\Brian chin\AppData\Local\Programs\Python\Python312\Lib\site-packages\rdflib\plugins\serializers\nt.py:41: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(



Knowledge graph creation complete:
- Entities: 4536
- Relationships: 24505
- Source documents: 1048

Generated files:
- TURTLE: Knowledge representation/knowledge_graphs\knowledge_representation_kg_20250624_121810.ttl
- JSONLD: Knowledge representation/knowledge_graphs\knowledge_representation_kg_20250624_121810.jsonld
- NTRIPLES: Knowledge representation/knowledge_graphs\knowledge_representation_kg_20250624_121810.nt
- SIMPLIFIED: Knowledge representation/knowledge_graphs\knowledge_representation_kg_simplified_20250624_121810.json

Knowledge graph created successfully!


# Relation Model Training
---

Create Train File

In [2]:
import json
from datetime import datetime
import os
from typing import List, Dict
from random import choice, sample

def generate_qa_pairs(json_file_path: str, output_dir: str, num_pairs_per_entity: int = 3) -> str:
    """
    Generates question-answer pairs from a JSON knowledge representation file.
    
    Args:
        json_file_path: Path to the JSON knowledge base file
        output_dir: Directory to save the output QA JSON file
        num_pairs_per_entity: Number of QA pairs to generate per entity
        
    Returns:
        Path to the generated QA JSON file
    """
    # Load the JSON knowledge base
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            knowledge_base = json.load(f)
    except Exception as e:
        raise ValueError(f"Failed to load JSON file: {str(e)}")
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate QA pairs
    qa_pairs = []
    
    # Question templates in Malay (Bahasa Malaysia)
    question_templates = [
        "Apakah {entity}?",
        "Boleh anda terangkan tentang {entity}?",
        "Apa yang anda tahu mengenai {entity}?",
        "Apakah kepentingan {entity} dalam budaya Cina Malaysia?",
        "Bagaimana {entity} berkaitan dengan budaya Cina Malaysia?",
        "Apakah makna {entity} dalam masyarakat Cina Malaysia?",
        "Boleh anda berikan maklumat tentang {entity}?",
        "Apakah peranan {entity} dalam tradisi Cina Malaysia?"
    ]
    
    # Process each document chunk to create QA pairs
    for chunk in knowledge_base:
        # Extract entities and relationships from triples
        entities = set()
        relationships = []
        
        for triple in chunk['triples']:
            if len(triple) != 3:
                continue
                
            subject = triple[0].strip()
            predicate = triple[1].strip()
            obj = triple[2].strip()
            
            if subject and obj:
                entities.add(subject)
                entities.add(obj)
                relationships.append((subject, predicate, obj))
        
        # Generate QA pairs for each entity
        for entity in entities:
            # Find relevant context from the original text
            context = find_entity_context(entity, chunk['text'])
            
            # Find answers that mention this entity
            possible_answers = []
            for subj, pred, obj in relationships:
                if entity in (subj, obj):
                    answer_text = f"{subj} {pred} {obj}"
                    possible_answers.append(answer_text)
            
            # If no direct relationships, use the context as answer
            if not possible_answers and context:
                possible_answers.append(context)
            
            # Generate multiple QA pairs per entity
            for _ in range(min(num_pairs_per_entity, len(question_templates))):
                if not possible_answers:
                    break
                    
                # Select a random question template and answer
                question_template = choice(question_templates)
                answer = choice(possible_answers)
                
                # Create the QA pair
                qa_pair = {
                    "question": question_template.format(entity=entity),
                    "context": context if context else "",
                    "answer": answer,
                    "language": "ms",
                    "topic": entity
                }
                
                qa_pairs.append(qa_pair)
    
    # Generate output filename
    base_filename = os.path.splitext(os.path.basename(json_file_path))[0]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = os.path.join(output_dir, f"qa_train.json")
    
    # Save the QA pairs
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(qa_pairs, f, indent=2, ensure_ascii=False)
    
    print(f"\nGenerated {len(qa_pairs)} QA pairs")
    print(f"Saved to: {output_path}")
    
    return output_path

def find_entity_context(entity: str, text: str) -> str:
    """
    Finds relevant context for an entity in the text.
    """
    sentences = text.split('.')
    for sentence in sentences:
        if entity in sentence:
            return sentence.strip() + '.'
    return ""

if __name__ == "__main__":
    # Example usage
    input_json = "Knowledge representation/knowledge_representation.json"
    output_dir = "Knowledge representation/qa_datasets"
    
    try:
        output_file = generate_qa_pairs(input_json, output_dir)
        print("\nQA dataset created successfully!")
    except Exception as e:
        print(f"\nError creating QA dataset: {str(e)}")
        import traceback
        traceback.print_exc()


Generated 27492 QA pairs
Saved to: Knowledge representation/qa_datasets\qa_train.json

QA dataset created successfully!


Model Training mBERT
<hr>

In [6]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForQuestionAnswering, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import os
import re
import string
from collections import Counter

# 1. Load data
print("Loading training data...")
with open("qa_train.json", encoding="utf-8") as f:
    qa_data = json.load(f)

print(f"Loaded {len(qa_data)} QA pairs")

# 2. Dataset class
class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item.get("question", "")
        context = item.get("context", question)
        answer = item.get("answer", "")

        if not question or not context or not answer:
            # Return a padded sample for empty data
            encoding = self.tokenizer(
                "empty", "empty",
                truncation=True,
                max_length=self.max_length,
                padding="max_length",
                return_offsets_mapping=True,
                return_tensors="pt"
            )
            encoding = {k: v.squeeze(0) for k, v in encoding.items()}
            encoding["start_positions"] = torch.tensor(0)
            encoding["end_positions"] = torch.tensor(0)
            encoding.pop("offset_mapping")
            return encoding

        # Find answer start and end positions in context
        start_idx = context.find(answer)
        if start_idx == -1:
            start_idx = 0  # fallback
        end_idx = start_idx + len(answer)

        encoding = self.tokenizer(
            question,
            context,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_offsets_mapping=True,
            return_tensors="pt"
        )

        # Find token-level start/end positions
        offsets = encoding["offset_mapping"][0]
        start_position, end_position = 0, 0
        for i, (start, end) in enumerate(offsets):
            if start <= start_idx < end:
                start_position = i
            if start < end_idx <= end:
                end_position = i

        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        encoding["start_positions"] = torch.tensor(start_position)
        encoding["end_positions"] = torch.tensor(end_position)
        encoding.pop("offset_mapping")
        return encoding

# 3. Evaluation functions
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    """Calculate F1 score between prediction and ground truth."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    """Calculate exact match score between prediction and ground truth."""
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def evaluate_model(model, tokenizer, data_loader, device, max_length=128):
    """Evaluate model and return EM and F1 scores."""
    model.eval()
    total_em = 0
    total_f1 = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Get predictions
            outputs = model(**batch)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits
            
            # Get predicted start and end positions
            start_preds = torch.argmax(start_logits, dim=-1)
            end_preds = torch.argmax(end_logits, dim=-1)
            
            # Convert predictions to text
            for i in range(len(start_preds)):
                start_pos = start_preds[i].item()
                end_pos = end_preds[i].item()
                
                # Get the original context and answer
                original_data = data_loader.dataset.data[total_samples + i]
                context = original_data.get("context", "")
                true_answer = original_data.get("answer", "")
                
                # Tokenize context to get offsets
                encoding = tokenizer(
                    original_data.get("question", ""),
                    context,
                    truncation=True,
                    max_length=max_length,
                    padding="max_length",
                    return_offsets_mapping=True,
                    return_tensors="pt"
                )
                offsets = encoding["offset_mapping"][0]
                
                # Extract predicted answer text
                if start_pos < len(offsets) and end_pos < len(offsets) and start_pos <= end_pos:
                    start_char = offsets[start_pos][0].item()
                    end_char = offsets[end_pos][1].item()
                    predicted_answer = context[start_char:end_char]
                else:
                    predicted_answer = ""
                
                # Calculate metrics
                em_score = exact_match_score(predicted_answer, true_answer)
                f1 = f1_score(predicted_answer, true_answer)
                
                total_em += em_score
                total_f1 += f1
            
            total_samples += len(start_preds)
    
    avg_em = total_em / total_samples if total_samples > 0 else 0
    avg_f1 = total_f1 / total_samples if total_samples > 0 else 0
    
    return avg_em, avg_f1

# 4. Split train/validation set
print("Splitting data into train/validation sets...")
train_data, val_data = train_test_split(qa_data, test_size=0.1, random_state=42)
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

# 5. Load mBERT tokenizer and model
print("Loading mBERT model and tokenizer...")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")

train_dataset = QADataset(train_data, tokenizer)
val_dataset = QADataset(val_data, tokenizer)

# Use num_workers=2 for better performance
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 6. Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 5
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Add mixed precision training for better performance
scaler = GradScaler()

# 7. Training loop with EM and F1 tracking
print(f"\nStarting training for {num_epochs} epochs...")
best_val_loss = float('inf')
best_em = 0
best_f1 = 0
training_history = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(**batch)
            loss = outputs.loss
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} average training loss: {avg_train_loss:.4f}")
    
    # Validation with EM and F1
    print("Running validation...")
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation Loss"):
            batch = {k: v.to(device) for k, v in batch.items()}
            with autocast():
                outputs = model(**batch)
                val_loss += outputs.loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    
    # Calculate EM and F1 scores
    print("Calculating EM and F1 scores...")
    em_score, f1_score_val = evaluate_model(model, tokenizer, val_loader, device)
    
    print(f"Epoch {epoch+1} Results:")
    print(f"  Validation Loss: {avg_val_loss:.4f}")
    print(f"  Exact Match: {em_score:.4f}")
    print(f"  F1 Score: {f1_score_val:.4f}")
    
    # Save training history
    epoch_results = {
        'epoch': epoch + 1,
        'train_loss': avg_train_loss,
        'val_loss': avg_val_loss,
        'em_score': em_score,
        'f1_score': f1_score_val
    }
    training_history.append(epoch_results)
    
    # Save best model based on F1 score (you can change this to EM if preferred)
    if f1_score_val > best_f1:
        best_f1 = f1_score_val
        best_em = em_score
        best_val_loss = avg_val_loss
        print(f"New best F1 score! Saving model...")
        model.save_pretrained("mbbert-qa-finetuned-best")
        tokenizer.save_pretrained("mbbert-qa-finetuned-best")
        
        # Save training history
        with open("training_history.json", "w", encoding="utf-8") as f:
            json.dump(training_history, f, indent=2)

# 8. Save final model
model.save_pretrained("mbbert-qa-finetuned")
tokenizer.save_pretrained("mbbert-qa-finetuned")
print("Model saved!")

Loading training data...
Loaded 610 QA pairs
Splitting data into train/validation sets...
Training samples: 549
Validation samples: 61
Loading mBERT model and tokenizer...


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Brian chin\AppData\Local\Temp\ipykernel_22852\3674536702.py:217: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Using device: cpu

Starting training for 5 epochs...


Epoch 1/5:   0%|          | 0/35 [00:00<?, ?it/s]C:\Users\Brian chin\AppData\Local\Temp\ipykernel_22852\3674536702.py:236: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/5: 100%|██████████| 35/35 [08:42<00:00, 14.93s/it, loss=0.6684]


Epoch 1 average training loss: 1.4484
Running validation...


...

Epoch 5 average training loss: 0.6017
Running validation...


Validation Loss: 100%|██████████| 4/4 [00:12<00:00,  3.03s/it]


Calculating EM and F1 scores...


Evaluating: 100%|██████████| 4/4 [00:12<00:00,  3.04s/it]


Epoch 5 Results:
  Validation Loss: 0.6632
  Exact Match: 0.0000
  F1 Score: 0.1346
Model saved!
